In [10]:
import pandas
df = pandas.read_csv('Region_Data.csv', encoding='gb2312', skiprows=3, skipfooter=2, engine = 'python')#跳过前面三行，后面三行

In [9]:
df.head()

,area,year,gross_product
0,北京市,2015,23014.59
1,天津市,2015,16538.19
2,河北省,2015,29806.11
3,山西省,2015,12766.49
4,内蒙古自治区,2015,17831.51


In [3]:
df = pandas.melt(df, col_level=0, id_vars='地区')#根据地区 转换数据

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
地区          310 non-null object
variable    310 non-null object
value       310 non-null float64
dtypes: float64(1), object(2)
memory usage: 7.3+ KB


In [5]:
df['variable'] = df['variable'].map(lambda e : int(e.strip('年')))#只留下年份的数字

In [6]:
df.columns = ['area', 'year', 'gross_product']#修改字段名

In [7]:
import sqlite3 as lite#写入数据库
with lite.connect('country_stat.sqlite') as db:#连接数据库
    df.to_sql('regional_gross_product', con = db, if_exists='replace', index=None)#不要index

In [11]:
import pandas#从数据库读取
with lite.connect('country_stat.sqlite') as db:
    df2 = pandas.read_sql('SELECT area, gross_product FROM regional_gross_product WHERE year = 2015', con = db)

In [12]:
df2.head()

,area,gross_product
0,北京市,23014.59
1,天津市,16538.19
2,河北省,29806.11
3,山西省,12766.49
4,内蒙古自治区,17831.51


In [96]:
import pandas#排序 生产总值
with lite.connect('country_stat.sqlite') as db:
    df3 = pandas.read_sql('SELECT * FROM regional_gross_product WHERE year = 2015 ORDER BY gross_product DESC LIMIT 3', con = db)
df3.head()

,area,year,gross_product
0,广东省,2015,72812.55
1,江苏省,2015,70116.38
2,山东省,2015,63002.33


In [13]:
import pandas#使用group by聚合数据，使用having挑选出平均生产总值为10000以上的地区
#AVG(gross_product)：平均生产总值
with lite.connect('country_stat.sqlite') as db:
    df4 = pandas.read_sql('SELECT area, AVG(gross_product) as avg_gross_product FROM regional_gross_product GROUP BY area HAVING avg_gross_product >= 10000 ORDER BY avg_gross_product DESC LIMIT 3', con = db)
df4.head()

,area,avg_gross_product
0,广东省,49403.249
1,江苏省,45275.185
2,山东省,42471.128
